In [158]:
import re
import time
import random
import string
import json
import requests

import pandas as pd

from datetime import datetime


# 配置请求参数
REQ_CONFIG = {
    'username': 's.mao@sungrow-emea.com',
    'password': 'QUcv8q8^77u#!',
    'appkey': 'A12BA0CD65EECFB33DC79841118C7268',
    'x_access_key': '4924f0931e50f51267433bd29225ed16',
    'domain': 'apieu.suncharger.cn'
}


# 获取当前UNIX时间戳（毫秒）
def current_timestamp():
    return int(time.time() * 1000)


# 生成32位随机字符串
def generate_nonce(length=32):
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


# 获取身份认证的token
def get_token(req_config:dict) -> str:
    # 获取请求配置参数
    username = req_config.get('username')
    password = req_config.get('password')
    appkey = req_config.get('appkey')
    x_access_key = req_config.get('x_access_key')
    api_domain = req_config.get('domain')
    # 生成必要数值
    url = f"https://{api_domain}/openapi/v1/auth"
    nonce = generate_nonce()
    timestamp = current_timestamp()
    # 请求体
    auth_request = {
        "nonce": nonce,
        "data": {
            "userAccount": username,
            "password": password
        },
        "timestamp": timestamp,
        "appkey": appkey,
        "lang": "_en_US"
    }
    # 请求头
    headers = {
        'Content-Type': 'application/json;charset=UTF-8',
        'x-client-tz': 'GMT+8',
        'x-access-key': x_access_key
    }
    # 响应
    response = requests.post(url, headers=headers, json=auth_request)
    
    if response.status_code == 200:
        print("Token received.")
        return response.json().get('data').get('token')
    else:
        print(f"Authorization failed: {response.text}")
        return None
    

# 查询单页场站列表
def get_station_list(token:str, req_config:dict, page:int) -> dict:
    """
    token: str
        token string returned by function get_token()
    req_config: dict
        request configuration
    page: int
        index of to be requested page
    return: dict
        json format data

    """
    # 获取请求配置参数
    appkey = req_config.get('appkey')
    x_access_key = req_config.get('x_access_key')
    api_domain = req_config.get('domain')
    # 生成必要数值
    url = f'https://{api_domain}/openapi/v1/getStationList'
    nonce = generate_nonce()
    timestamp = str(int(time.time() * 1000))
    # 请求体
    station_request = {
        'nonce': nonce,
        'data': {
            'curPage': page,
            'size': 1000,
            #'stationStatus': 'WORKING'
        },
        'timestamp': timestamp,
        'token': token,
        'appkey': appkey,
        'lang': '_en_US'
    }
    # 请求头
    headers = {
        'Content-Type': 'application/json;charset=UTF-8',
        'x-client-tz': 'GMT+8',
        'x-access-key': x_access_key
    }
    # 反馈
    response = requests.post(url, headers=headers, json=station_request)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"查询场站列表失败: {response.text}")
        return None
    

# 记录所有场站列表
def fetch_station_data(token:str, req_config:dict, page:int=1, df:pd.DataFrame=None) -> pd.DataFrame:
    # Initial call to create an empty DataFrame
    if df is None:
        df = pd.DataFrame()
    print(f"\rProcessing page {page}...", end='')
    # 获取每一页面的站点列表
    station_list_response = get_station_list(token, req_config, page)
    # 将当前页面的站点数据转换为 DataFrame
    current_list = station_list_response.get('data').get('list', [])
    new_df = pd.DataFrame(current_list)
    # 合并数据
    df = pd.concat([df, new_df], ignore_index=True)
    # 检查是否还有下一页
    if station_list_response.get('data').get('hasNextPage'):
        # 准备下一页的请求参数
        return fetch_station_data(token, req_config, page+1, df)
    else:
        print("\nFinal page reached.")
        # 返回最终合并的 DataFrame
        return df
    

# 筛选特定名称的站点
def filter_station_data(df: pd.DataFrame) -> pd.DataFrame:
    # 使用正则表达式筛选 stationName 以 EVN 开头的数据
    filtered_df = df[df['stationName'].str.match(r'^EVN.*', na=False)]
    # 定义处理函数
    def verify_location_and_country(row):
        # 正则表达式匹配 location 的格式
        match = re.match(r'^(.*),\s*(\d{4})\s*(\w.+),\s*(.+)$', row['location'])
        if match:
            # 更新 location 和 countryName 的值
            row['location'] = match.group(1)  # 街道地址
            row['countryName'] = match.group(4)  # 国家名称
        return row
    # 应用处理函数
    filtered_df = filtered_df.apply(verify_location_and_country, axis=1)
    
    return filtered_df[['stationId', 'stationName', 'location', 'postalCode', 'city', 'countryName', 'longitude', 'latitude', 'stationStatus']]


# 检查DataFrame中的空值
def result_df_check(df:pd.DataFrame):
    # 打印信息总览
    #df.info()
    # 记录总行数
    total_entries = len(df)
    # 用于存储结果的字典
    #result = {}
    # 检查每一列是否存在非空
    for column in df.columns:
        non_null_count = df[column].notnull().sum()  # 计算非空条目数量
        if non_null_count < total_entries:
            # 如果非空数量少于总数量，筛选出该列值为 null 的条目
            null_entries_df = df[df[column].isnull()]
            print("\nNull in column:")
            print(null_entries_df[['stationName', column]])
    # 检查经纬度值
    longlat_zero = df[(df['longitude']==0 )|(df['latitude']==0)]
    if len(longlat_zero) != 0:
        print("\nZero in column:")
        print(longlat_zero[['stationName', 'longitude', 'latitude']])
    return



In [ ]:

# 
def get_orders_for_stations(token:str, req_config:dict, station_df:pd.DataFrame, 
                            start_time:str=datetime.now().replace(month=1, day=1).strftime("%Y-%m-%d"), 
                            end_time:str=datetime.now().strftime("%Y-%m-%d")) -> pd.DataFrame:

    # 获取请求配置参数
    appkey = req_config.get('appkey')
    x_access_key = req_config.get('x_access_key')
    api_domain = req_config.get('domain')
    # 生成必要数值
    url = f'https://{api_domain}/openapi/v1/getStationList'
    nonce = generate_nonce()
    timestamp = str(int(time.time() * 1000))

    all_orders_df = pd.DataFrame()  # 存储所有订单的数据

    for index, row in station_df.iterrows():
        station_id = row['stationId']
        station_name = row['stationName']  # 这里可以用来调试或记录
        
        # 构建请求参数
        request_data = {
            "nonce": "random_nonce",  # 实际应用中生成随机的nonce
            "data": {
                "stationId": station_id,
                "orderStatus": "Closed",  # 根据要求的订单状态
                "startTime": start_time,
                "endTime": end_time,
            },
            "timestamp": int(time.time() * 1000),  # 当前时间戳（毫秒）
            "token": TOKEN,
            "appkey": APPKEY,
            "lang": "_en_US"
        }

        # 发送请求
        response = requests.post(f"https://{API_HOST}/openapi/v1/getOrderList", json=request_data)
        
        if response.status_code == 200:  # 检查请求是否成功
            response_data = response.json()
            if response_data.get('ok'):
                order_list = response_data['data'].get('orderList', [])
                
                if order_list:
                    # 将 orders 转换为 DataFrame 并添加 stationId 列
                    orders_df = pd.DataFrame(order_list)
                    orders_df['stationId'] = station_id  # 添加 stationId 列
                    orders_df['stationName'] = station_name  # 可选：添加 stationName 列
                    
                    # 合并到总的 DataFrame
                    all_orders_df = pd.concat([all_orders_df, orders_df], ignore_index=True)
            else:
                print(f"Error fetching orders for station {station_id}: {response_data.get('message')}")
        else:
            print(f"API request failed with status code {response.status_code}")

    return all_orders_df


In [159]:
# 主程序
if __name__ == "__main__":

    token = get_token(REQ_CONFIG)

    if token:
        df = fetch_station_data(token, REQ_CONFIG, 1)
        # 输出最终的 DataFrame
        print(f"{len(df)} total stations fetched.")
    else:
        print("Token unavailable.")

    result_df = filter_station_data(df)

    result_df_check(result_df)




Token received.
Processing page 17...
Final page reached.
16271 total stations fetched.

Null in column:
                     stationName postalCode
308  EVN - Salzburg - Fürbergstr       None

Zero in column:
                     stationName  longitude  latitude
308  EVN - Salzburg - Fürbergstr        0.0       0.0


In [157]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 66 to 16210
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   stationId      73 non-null     object 
 1   stationName    73 non-null     object 
 2   location       73 non-null     object 
 3   postalCode     72 non-null     object 
 4   city           73 non-null     object 
 5   countryName    73 non-null     object 
 6   longitude      73 non-null     float64
 7   latitude       73 non-null     float64
 8   stationStatus  73 non-null     object 
dtypes: float64(2), object(7)
memory usage: 5.7+ KB
